In [302]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split

In [303]:
games_df = pd.read_csv('games.csv')
games_details_df = pd.read_csv('games_details.csv')
players_df = pd.read_csv('players.csv')
ranking_df = pd.read_csv('ranking.csv')
teams_df = pd.read_csv('teams.csv')

C:\Users\fergb\AppData\Local\Temp\ipykernel_35100\2321718875.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details_df = pd.read_csv('games_details.csv')


In [304]:
games_details_df = games_details_df[games_details_df['COMMENT'].isna()]

In [305]:
def min_to_float(min):
    if (type(min) == str):
        if (':' in min):
            if ("." in min):
                split = min.split('.')
                min = ":".join([split[0], split[1][-2:]])
            mins, secs = map(int, min.split(':'))
            return (mins + secs) / 60.0
        else:
            return float(min)
    return min

games_details_df["MIN"] = games_details_df["MIN"].apply(min_to_float)

In [306]:
game_dropped_columns = ["GAME_DATE_EST", "GAME_ID" , "GAME_STATUS_TEXT", "HOME_TEAM_ID", "VISITOR_TEAM_ID"]
details_dropped_columns = ["GAME_ID","TEAM_ID","TEAM_ABBREVIATION","TEAM_CITY","PLAYER_ID","PLAYER_NAME","NICKNAME","START_POSITION", 'COMMENT']

In [307]:
def get_team_id(name):
    return teams_df[teams_df['NICKNAME'] == name]['TEAM_ID'].values[0]

In [308]:
def predict_classifer(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X,              
                                                    y,              
                                                    test_size=0.3,  
                                                    random_state=7,
                                                    stratify=y
                                                   )
    rf = RandomForestClassifier(random_state=7)
    rf.fit(X_train, y_train)

    return rf.predict(X_test)

In [309]:
def predict_regressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X,              
                                                    y,              
                                                    test_size=0.3,  
                                                    random_state=7,
                                                
                                                   )
    rf = RandomForestRegressor(random_state=7)
    rf.fit(X_train, y_train)

    return rf.predict(X_test)

In [310]:
def predict_winner(home, away):
    X = games_df[(games_df["HOME_TEAM_ID"] == get_team_id(home)) & (games_df['VISITOR_TEAM_ID'] == get_team_id(away))]
    X.drop(columns=game_dropped_columns)
    y = X['HOME_TEAM_WINS']
    X = X.drop(columns='HOME_TEAM_WINS')
    
    predictions = predict_classifer(X, y)

    result = np.bincount(predictions).argmax()
    
    if (result == 1):
        print(home + ' win')
    else:
        print(away + ' win')

In [311]:
def predict_score(home, away):
    X = games_df[(games_df["HOME_TEAM_ID"] == get_team_id(home)) & (games_df['VISITOR_TEAM_ID'] == get_team_id(away))]
    X = X.drop(columns=game_dropped_columns)
    y = X[['PTS_home', 'PTS_away']]
    X = X.drop(columns=['PTS_home', 'PTS_away'])

    predictions = predict_regressor(X, y)

    home_scores = predictions[:, 0]
    away_scores = predictions[:, 1]

    print(home + ' will score ' + str(np.mean(home_scores).round(0)))
    print(away + ' will score ' + str(np.mean(away_scores).round(0)))


In [329]:
def predict_player_stat_against_team(player, team, stat):
    games_played_by_player = games_details_df[games_details_df['PLAYER_NAME'] == player]
    game_ids = games_played_by_player['GAME_ID'].tolist()
    
    games_ids_against_team = []
    for game_id in game_ids:
        game = games_df[games_df['GAME_ID'] == game_id]
        if (game["HOME_TEAM_ID"].values[0] == get_team_id(team) or game["VISITOR_TEAM_ID"].values[0] == get_team_id(team)):
            games_ids_against_team.append(game_id)

    games_against_team = games_played_by_player[games_played_by_player['GAME_ID'].isin(games_ids_against_team)]
    X = games_against_team.drop(columns=details_dropped_columns)
    y = X[stat]
    X = X.drop(columns=stat)

    predictions = predict_regressor(X, y)

    print(player + ' will get '  + str(np.mean(predictions).round(0)) + " " + stat)

In [330]:
predict_player_stat_against_team('Bradley Beal', 'Celtics', 'REB')

Bradley Beal will get 4.0 REB
